# **Tarea 3 - Word Embeddings 📚**

**Integrantes:**

**Fecha límite de entrega 📆:** Martes 06 de Julio.

**Tiempo estimado de dedicación:**


**Instrucciones:**
- El ejercicio consiste en:
    - Responder preguntas relativas a los contenidos vistos en los vídeos y slides de las clases. 
    - Entrenar Word2Vec y FastText sobre un pequeño corpus.
    - Evaluar los embeddings obtenidos en una tarea de clasificación.
- La tarea se realiza en grupos de **máximo** 2 personas. Puede ser invidivual pero no es recomendable.
- La entrega es a través de u-cursos a más tardar el día estipulado arriba. No se aceptan atrasos.
- El formato de entrega es este mismo Jupyter Notebook.
- Al momento de la revisión tu código será ejecutado. Por favor verifica que tu entrega no tenga errores de compilación. 
- En el horario de auxiliar pueden realizar consultas acerca de la tarea a través del canal de Discord del curso. 


**Referencias**

Vídeos: 

- [Linear Models](https://youtu.be/zhBxDsNLZEA)
- [Neural Networks](https://youtu.be/oHZHA8h2xN0)
- [Word Embeddings](https://youtu.be/wtwUsJMC9CA)

## **Preguntas teóricas 📕 (3 puntos).** ##
Para estas preguntas no es necesario implementar código, pero pueden utilizar pseudo código.

### **Parte 1: Modelos Lineales (1.5 ptos)**

Suponga que tiene un dataset de 10.000 documentos etiquetados por 4 categorías: política, deporte, negocios y otros. 

**Pregunta 1**: Diseñe un modelo lineal capaz de clasificar un documento según estas categorías donde el output sea un vector con una distribución de probabilidad con la pertenencia a cada clase. 

Especifique: representación de los documentos de entrada, parámetros del modelo, transformaciones necesarias para obtener la probabilidad de cada etiqueta y función de pérdida escogida. **(0.75 puntos)**

**Respuesta**: Representación escogida del documento de entrada: Bag of words por ejemplo

Parámetros del modelo: Matriz de pesos W en donde la columna 1 tengan mayor importancia las palabras de política, columna 2 sobre deporte, columna 3 sobre negocios y columna 4 el resto

Transformaciones necesarias: Aplicar softmax al vector de output del modelo.

Función de pérdida escogida: Cross-entropy es el ideal, pero pueden escoger cualquier función de perdida multiclase que alcanze el mínimo cuando las predicciones son correctas


**Pregunta 2**: Explique cómo funciona el proceso de entrenamiento en este tipo de modelos y su evaluación. **(0.75 puntos)**

**Respuesta**: El objetivo del entrenamiento es minimizar la loss del modelo. Se van tuneando los parámetros de la matriz W hasta encontrar uno que minimice la loss del modelo. La evaluación se hace sobre datos que no se han observado para comprobar generalización del modelo.

### **Parte 2: Redes Neuronales (1.5 ptos)** 

Supongamos que tenemos la siguiente red neuronal.

![image.png](https://drive.google.com/uc?export=view&id=1fFTjtMvH6MY8o42_vj010y8eTuCVb5a3)

**Pregunta 1**: En clases les explicaron como se puede representar una red neuronal de una y dos capas de manera matemática. Dada la red neuronal anterior, defina la salida $\vec{\hat{y}}$ en función del vector $\vec{x}$, pesos $W^i$, bias $b^i$ y funciones $g,f,h$. 

Adicionalmente liste y explicite las dimensiones de cada matriz y vector involucrado en la red neuronal. **(0.75 Puntos)**

**Respuesta**: 

Formula:
$\vec{\hat{y}} = NN_{MLP3}(\vec{x})= h(f(g(\vec{x}W¹+\vec{b}¹)W²+\vec{b}²)W³+\vec{b}³)W⁴+\vec{b}⁴$ (1 punto)

Dimensiones: (2 puntos)
- $\vec{x}$: 3
- $W¹$: 3x2
- $\vec{b}¹$: 2
- $W²$: 2x3
- $\vec{b}²$: 3
- $W³:$ 3x1
- $\vec{b}³:$ 1
- $W⁴:$ 1x4
- $\vec{b}⁴:$ 4

**Pregunta 2**: Explique qué es backpropagation. ¿Cuales serían los parámetros a evaluar en la red neuronal anterior durante backpropagation? **(0.25 puntos)**

**Respuesta**: "Backpropagation es una técnica eficiente para evaluar el gradiente de una loss function L en una red neuronal feed-forward con respecto a todos sus parámetros" (Cita de clases). Los parámetros serian de $W¹,\vec{b}¹$ a $W⁴,\vec{b}⁴$

**Pregunta 3**: Explique los pasos de backpropagation. En la red neuronal anterior: Cuales son las derivadas que debemos calcular para poder obtener $\vec{\delta^l_{[j]}}$ en todas las capas? **(0.5 puntos)**

**Respuesta**: Los 4 pasos son:

- Aplicar el vector x y propagarlo por toda la red
- Evaluar $\delta$ para todas las unidades ocultas
- Propagar los $\delta$ desde el final al inicio de la red
- Ocupar la formula de $\frac{\partial L}{\partial W}$

En la red anterior se necesitan las derivadas: $f', g',h'$

## **Preguntas prácticas 💻 (3 puntos).** ##

### **Parte 3: Word Embeddings**

En la auxiliar 2 aprendieron como entrenar Word2Vec utilizando gensim. El objetivo de esta parte es comparar los embeddings obtenidos con dos modelos diferentes: Word2Vec y [FastText](https://radimrehurek.com/gensim/models/fasttext.html) (utilizen size=200 en FastText) entrenados en el mismo dataset de diálogos de los Simpson. 

In [ ]:
import re  
import pandas as pd 
from time import time  
from collections import defaultdict 
import string 
import multiprocessing
import os
import gensim
import sklearn
from sklearn import linear_model
from collections import Counter
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, classification_report

# word2vec
from gensim.models import Word2Vec, KeyedVectors, FastText
from gensim.models.phrases import Phrases, Phraser
from sklearn.model_selection import train_test_split
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

Utilizando el dataset adjunto con la tarea:

In [ ]:
data_file = "dialogue-lines-of-the-simpsons.zip"
df = pd.read_csv(data_file)
stopwords = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/english.txt'
).values
stopwords = Counter(stopwords.flatten().tolist())
df = df.dropna().reset_index(drop=True) # Quitar filas vacias

2021-05-05 17:58:59,568 : INFO : NumExpr defaulting to 2 threads.


**Pregunta 1**: Ayudándose de los pasos vistos en la auxiliar, entrene los modelos Word2Vec y FastText sobre el dataset anterior. **(1 punto)** (Hint, le puede servir explorar un poco los datos)

**Respuesta**:

In [ ]:
punctuation = string.punctuation + "«»“”‘’…—"
def simple_tokenizer(doc, lower=False):
    if lower:
        tokenized_doc = doc.translate(str.maketrans(
            '', '', punctuation)).lower().split()

    tokenized_doc = doc.translate(str.maketrans('', '', punctuation)).split()

    tokenized_doc = [
        token for token in tokenized_doc if token.lower() not in stopwords
    ]
    return tokenized_doc
content = df['spoken_words']
cleaned_content = [simple_tokenizer(doc) for doc in content.values]

In [ ]:
w2v_model = Word2Vec(min_count=10,
                      window=4,
                      size=200,
                      sample=6e-5,
                      alpha=0.03,
                      min_alpha=0.0007,
                      negative=20,
                      workers=multiprocessing.cpu_count())
ft_model = FastText(size=200, window=3, min_count=1)

In [ ]:
w2v_model.build_vocab(sentences, progress_per=10000)
ft_model.build_vocab(sentences, progress_per=10000)

In [ ]:
t = time()
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=15, report_delay=10)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))
if not os.path.exists('./pretrained_models'):
    os.mkdir('./pretrained_models')
w2v_model.save('./pretrained_models/w2v.model')

In [ ]:
t = time()
ft_model.train(sentences, total_examples=ft_model.corpus_count, epochs=15, report_delay=10)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))
if not os.path.exists('./pretrained_models'):
    os.mkdir('./pretrained_models')
ft_model.save('./pretrained_models/ft.model')

**Pregunta 2**: Encuentre las palabras mas similares a las siguientes: Lisa, Bart, Homer, Marge. Cúal es la diferencia entre ambos resultados? Por qué ocurre esto? Intente comparar ahora Liisa en ambos modelos (doble i). Cuando escogería uno vs el otro? **(0.5 puntos)**

**Respuesta**:

In [ ]:
w2v_model.wv.most_similar(positive=["Lisa"])

In [ ]:
ft_model.wv.most_similar(positive=["Lisa"])

In [ ]:
w2v_model.wv.most_similar(positive=["Bart"])

In [ ]:
ft_model.wv.most_similar(positive=["Bart"])

In [ ]:
w2v_model.wv.most_similar(positive=["Homer"])

In [ ]:
ft_model.wv.most_similar(positive=['Homer'])

In [ ]:
w2v_model.wv.most_similar(positive=["Marge"])

In [ ]:
ft_model.wv.most_similar(positive=["Marge"])

In [ ]:
w2v_model.wv.most_similar(positive=["Liisa"]) # Es normal el siguiente error!

In [ ]:
ft_model.wv.most_similar(positive=["Liisa"])

W2V encuentra las palabras del vocabulario que están mas cerca mientras que FastText busca los ngramas. FastText funciona mejor que W2V cuando buscamos palabras que están fuera del vocabulario.


### **Parte 4: Aplicar embeddings para clasificar**

Ahora utilizaremos los embeddings que acabamos de calcular para clasificar palabras basadas en su polaridad (positivas o negativas). 

Para esto ocuparemos el lexicón AFINN incluido en la tarea, que incluye una lista de palabras y un 1 si su connotación es positiva y un -1 si es negativa.

In [ ]:
AFINN = 'AFINN_full.csv'
df_afinn = pd.read_csv(AFINN, sep='\t', header=None)

Hint: Para w2v son esperables KeyErrors debido a que no todas las palabras del corpus de los simpsons tendrán una representación en AFINN. Pueden utilizar esta función auxiliar para filtrar las filas en el dataframe que no tienen embeddings (como w2v no tiene token UNK se deben ignorar).

In [ ]:
def try_apply(model,word):
    try:
        aux = model[word]
        return True
    except KeyError:
        #logger.error('Word {} not in dictionary'.format(word))
        return False

**Pregunta 1**: Transforme las palabras del corpus de AFINN a la representación en embedding que acabamos de calcular (con ambos modelos). 

Su dataframe final debe ser del estilo [embedding, sentimiento], donde los embeddings corresponden a $X$ y el sentimiento asociado con el embedding a $y$ (positivo/negativo, 1/-1). 

Para ambos modelos, separar train y test de acuerdo a la siguiente función. **(0.75 puntos)**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=y)

**Respuesta**:

In [ ]:
df_afinn = df_afinn[df_afinn[0].apply(lambda x: try_apply(ft_model.wv,x))]
df_afinn[0] = df_afinn[0].apply(lambda x: ft_model.wv[x])
X = np.stack(df_afinn[0].values)
y = df_afinn[1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=y)

**Pregunta 2**: Entrenar una regresión logística (vista en auxiliar) y reportar accuracy, precision, recall, f1 y confusion_matrix para ambos modelos. Por qué se obtienen estos resultados? Cómo los mejorarías? **(0.75 puntos)**

**Respuesta**:

In [ ]:
reg = linear_model.LogisticRegression(penalty='l2', solver='liblinear', C=1)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
acc = accuracy_score(y_test, y_pred)
pre = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf = confusion_matrix(y_test, y_pred)

In [ ]:
logger.info("The accuracy is {0}".format(acc))
logger.info("The precision is {0}".format(pre))
logger.info("The recall is {0}".format(rec))
logger.info("The f1 score is {0}".format(f1))
logger.info("The confusion matrix:\n{0}".format(conf))

In [ ]:
df_afinn = pd.read_csv(AFINN, sep='\t', header=None)
df_afinn = df_afinn[df_afinn[0].apply(lambda x: try_apply(w2v_model,x))]
df_afinn[0] = df_afinn[0].apply(lambda x: w2v_model[x])
X = np.stack(df_afinn[0].values)
y = df_afinn[1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=y)
reg = linear_model.LogisticRegression(penalty='l2', solver='liblinear', C=1)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
acc = accuracy_score(y_test, y_pred)
pre = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf = confusion_matrix(y_test, y_pred)
logger.info("The accuracy is {0}".format(acc))
logger.info("The precision is {0}".format(pre))
logger.info("The recall is {0}".format(rec))
logger.info("The f1 score is {0}".format(f1))
logger.info("The confusion matrix:\n{0}".format(conf))

Estos resultados son pésimos. Hay 2 razones posibles: El dataset es muy pequeño cómo para que estos modelos logren aprender bien las relaciones entre las palabras o puede que con los diálogos de los Simpson no se obtengan buenos embeddings para clasificar palabras por sentimiento. Se podría mejorar añadiendo mas datos, por ejemplo los subtítulos de las películas o simplemente buscar otro dataset más grande.

# Bonus: +0.25 puntos en cualquier pregunta

**Pregunta 1**: Replicar la parte anterior utilizando embeddings pre-entrenados en un dataset más grande y obtener mejores resultados. Les puede servir [ésta](https://radimrehurek.com/gensim/downloader.html#module-gensim.downloader) documentacion de gensim **(0.25 puntos)**.

**Respuesta**:

In [ ]:
df_afinn = pd.read_csv(AFINN, sep='\t', header=None)
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-300") 
df_afinn = df_afinn[df_afinn[0].apply(lambda x: try_apply(model,x))]
df_afinn[0] = df_afinn[0].apply(lambda x: model[x])
X = np.stack(df_afinn[0].values)
y = df_afinn[1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=y)
reg = linear_model.LogisticRegression(penalty='l2', solver='liblinear', C=1)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
acc = accuracy_score(y_test, y_pred)
pre = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf = confusion_matrix(y_test, y_pred)
logger.info("The accuracy is {0}".format(acc))
logger.info("The precision is {0}".format(pre))
logger.info("The recall is {0}".format(rec))
logger.info("The f1 score is {0}".format(f1))
logger.info("The ROC AUC score is {0}".format(roc))
logger.info("The confusion matrix:\n{0}".format(conf))
logger.info("The kappa is {0}".format(kappa))
logger.info("Summary report:\n{0}".format(class_rep))